In [17]:
# prompt: Import libraries and open CSV

import pandas as pd
import numpy as np
import re
import datetime
import matplotlib.pyplot as plt


In [18]:
# Load the pickled DataFrame
# complete_data = pd.read_pickle("C:\\Users\\dfber\\OneDrive - Mass General Brigham\\Epidural project\\Data\\processed_merlin_data.pkl")
complete_data = pd.read_pickle("C:\\Users\\User\\OneDrive - Mass General Brigham\\Epidural project\\Data\\processed_merlin_data.pkl")

# Now you can work with the DataFrame
complete_data.head()

,anes_procedure_encounter_id_2273,id,epic_pmrn,delivery_date,delivery_time,pregnancy_outcome,newborn_gestational_age,gestational_age_2052,anesthesia_epidural_2062,anesthesia_general_2063,...,has_back_problems,maternal_race,composite_psychosocial_problems,any_public_insurance,maternal_language_english,marital_status_married_or_partner,country_of_origin_USA,employment_status_fulltime,epidural_needle_type,paresthesias_present
112804,3080063638,103917,10041002948,2015-07-29,16:01:11-04:00,term,277.0,277.0,True,False,...,False,Black,False,True,True,False,False,False,other,False
61731,3080177786,56802,10093675293,2015-07-30,15:40:00-04:00,term,289.0,289.0,True,False,...,False,Other/Unknown,False,False,True,False,False,False,other,False
78152,3080061118,71970,10047033192,2015-07-29,13:54:00-04:00,term,274.0,274.0,True,False,...,False,White,False,False,True,True,True,True,other,False
53664,3080229183,49385,10021864456,2015-07-29,02:39:00-04:00,term,261.0,261.0,True,False,...,False,Black,False,False,True,False,False,True,other,False
118656,3080183318,109315,10054753336,2015-07-29,02:47:27-04:00,term,293.0,293.0,True,False,...,False,White,False,False,True,True,True,False,other,False


In [19]:
df = complete_data.copy()

# Manually analyze some successes and failures

In [20]:
raw_identified_data = pd.read_csv("C:\\Users\\User\OneDrive - Mass General Brigham\\Epidural project\\Data\\Full Identified raw anesthesia_procedure_notes.csv")

<>:1: SyntaxWarning: invalid escape sequence '\O'
<>:1: SyntaxWarning: invalid escape sequence '\O'
C:\Users\User\AppData\Local\Temp\ipykernel_22472\3630446683.py:1: SyntaxWarning: invalid escape sequence '\O'
  raw_identified_data = pd.read_csv("C:\\Users\\User\OneDrive - Mass General Brigham\\Epidural project\\Data\\Full Identified raw anesthesia_procedure_notes.csv")


In [21]:
# Create a mapping of 'NoteID' to 'epic_pmrn' from raw_identified_data
note_to_mrn = raw_identified_data.set_index('NoteID')['epic_pmrn'].to_dict()
note_to_purpose = raw_identified_data.set_index('NoteID')['NotePurposeDSC'].to_dict()

# Use the mapping to create the new 'mrn' column in df
df['epic_pmrn'] = df['anes_procedure_note_id_2260'].map(note_to_mrn)
df['NotePurposeDSC'] = df['anes_procedure_note_id_2260'].map(note_to_purpose)

In [22]:
# prompt: Choose 10 random failed_catheters and 10 random non-failed_catheters

# Assuming 'df' is your DataFrame and it contains a column 'failed_catheter'
failed_catheters = df[df['failed_catheter'] == 1]
non_failed_catheters = df[df['failed_catheter'] == 0]

# Randomly choose 10 failed catheters
random_failed_catheters = failed_catheters.sample(n=10, random_state=42)  # random_state for reproducibility
chosen_failed_catheter_encounter_ids = ['3324914343','3272008150','3234765502','3305371022','3216449190','3186345033','3493903332','3285273066','3320528828','3191160118']
chosen_failed_catheters = df[df['anes_procedure_encounter_id_2273'].isin(chosen_failed_catheter_encounter_ids)]

# Randomly choose 10 non-failed catheters
random_non_failed_catheters = non_failed_catheters.sample(n=10, random_state=42) # random_state for reproducibility

In [23]:
column_names = [
    'epic_pmrn',
    "best_timestamp",
    "failed_catheter",
    "true_procedure_type",
    "NotePurposeDSC",
    "Regulated_Anesthesiologist_Name",
    "Regulated_Resident_Name",
    "anes_procedure_encounter_id_2273",
    "anes_procedure_note_id_2260",
    "subsequent_proof_of_failure_note_id",
]

In [24]:
random_failed_catheters[column_names]

,epic_pmrn,best_timestamp,failed_catheter,true_procedure_type,NotePurposeDSC,Regulated_Anesthesiologist_Name,Regulated_Resident_Name,anes_procedure_encounter_id_2273,anes_procedure_note_id_2260,subsequent_proof_of_failure_note_id
84393,10091501105,2019-11-20 06:30:00+00:00,True,epidural,ADDENDUM,Chang Liu,Evan Hodell,3270860547,2966490220,['2966955064']
6420,10033885699,2020-02-07 19:00:00+00:00,True,epidural,NORMAL,NaN,NaN,3284156048,3351887998,['3353863837']
118527,10042360866,2021-09-19 14:08:00+00:00,True,intrathecal,ADDENDUM,Rebecca Minehart,NaN,3386446141,6152717915,['6153238506']
1210,10146198345,2022-08-13 10:06:00+00:00,True,epidural,NORMAL,Holly Careskey,NaN,3452010786,7915985917,['7923223908']
46085,10094020087,2017-04-22 13:29:00+00:00,True,epidural,ADDENDUM,Lisa Leffert,Fernan Cuadra,3147096491,1417716364,['1417827464']
75258,10152223920,2023-07-18 04:34:00+00:00,True,epidural,NORMAL,Jimin Kim,Eduar Tole,3522692167,9793207509,['9793289265']
137027,10049722183,2023-02-13 15:53:00+00:00,True,epidural,NORMAL,Jimin Kim,Qanetha Ahmed,3489702423,8920877468,['8921877732']
24480,10068657096,2020-04-09 21:17:00+00:00,True,epidural,ADDENDUM,NaN,NaN,3294059792,3669778784,['3669814506']
132136,10091399435,2024-05-25 00:34:00+00:00,True,epidural,NORMAL,Ayumi Maeda,Ahmed Yousef,3594358424,11619219927,"['11619448867', '11626906547']"
8929,10130319121,2020-04-08 19:22:00+00:00,True,epidural,NORMAL,Mihaela Podovei,NaN,3294027209,3668910622,['3668913841']


In [25]:
chosen_failed_catheters[column_names]

,epic_pmrn,best_timestamp,failed_catheter,true_procedure_type,NotePurposeDSC,Regulated_Anesthesiologist_Name,Regulated_Resident_Name,anes_procedure_encounter_id_2273,anes_procedure_note_id_2260,subsequent_proof_of_failure_note_id
90428,10077811080,2018-03-19 06:39:00+00:00,True,epidural,ADDENDUM,Tong-Yan Chen,Kayla Florio,3186345033,1827698916,['1827810492']
90427,10077811080,2018-03-19 08:17:00+00:00,False,cse,ADDENDUM,Tong-Yan Chen,NaN,3186345033,1827810492,None
33948,10122458838,2018-04-26 14:12:00+00:00,False,spinal,NORMAL,Mihaela Podovei,NaN,3191160118,1875519865,None
33949,10122458838,2018-04-26 15:57:00+00:00,True,epidural,NORMAL,Mihaela Podovei,NaN,3191160118,1875532828,['1875559634']
33947,10122458838,2018-04-26 19:24:00+00:00,False,epidural,NORMAL,Mihaela Podovei,NaN,3191160118,1875559634,None
39816,10030249329,2018-11-06 18:26:00+00:00,True,epidural,NORMAL,NaN,NaN,3216449190,2126903072,"['2127240380', '2127290497']"
39817,10030249329,2018-11-07 05:35:00+00:00,True,epidural,NORMAL,Laura Chang,NaN,3216449190,2127240380,['2127290497']
39814,10030249329,2018-11-07 07:39:00+00:00,False,cse,NORMAL,Laura Chang,NaN,3216449190,2127290497,None
26063,10127925815,2019-03-25 05:11:00+00:00,True,epidural,NORMAL,Eric Cappiello,Yun-Yun Chen,3234765502,2302729884,['2306197611']
26064,10127925815,2019-03-27 08:45:00+00:00,False,epidural,NORMAL,Naida Cole,NaN,3234765502,2306197611,None


In [26]:
random_non_failed_catheters[column_names]

,epic_pmrn,best_timestamp,failed_catheter,true_procedure_type,NotePurposeDSC,Regulated_Anesthesiologist_Name,Regulated_Resident_Name,anes_procedure_encounter_id_2273,anes_procedure_note_id_2260,subsequent_proof_of_failure_note_id
69552,10049972754,2021-08-03 14:18:00+00:00,False,spinal,NORMAL,Kyle Jespersen,Grace Donzelli,3377173873,5919967912,None
80821,10081077975,2021-12-13 10:12:00+00:00,False,cse,ADDENDUM,Kyle Jespersen,Sarvesh Kaushik,3402994528,6613358148,None
4701,10120425581,2018-11-12 11:49:00+00:00,False,airway,NORMAL,NaN,NaN,3216724048,2133937860,None
81976,10060441838,2024-01-19 19:00:00+00:00,False,cse,ADDENDUM,Emily Naoum,Jessica Hawkins,3566056644,10868429844,None
23453,10094349676,2016-07-02 04:46:00+00:00,False,cse,NORMAL,Michael Murphy,NaN,3117290650,1006316042,None
108769,10096906184,2019-09-10 22:48:00+00:00,False,epidural,NORMAL,Kyle Byrne,NaN,3260150362,2545874580,None
61947,10141750223,2020-06-28 23:11:00+00:00,False,epidural,NORMAL,Johanna O'Connor,NaN,3305031394,3966339612,None
110431,10064481590,2019-10-05 04:58:00+00:00,False,epidural,NORMAL,NaN,NaN,3263564688,2711187425,None
120956,10032034299,2022-09-11 14:04:00+00:00,False,spinal,NORMAL,Lawrence Tsen,Ryan Sica,3457752268,8070560164,None
16127,10085465994,2016-03-07 23:23:00+00:00,False,epidural,NORMAL,NaN,Hoi Chan,3103840614,751800611,None


In [ ]:
df[df['anes_procedure_encounter_id_2273'] == '3191160118'][column_names]

In [13]:
# Filter the DataFrame for failed catheters and delivery location 'mgh'
random_failed_catheters_mgh = df[(df['failed_catheter'] == True) & (df['delivery_site_2188'] == 'mgh')].sample(n=10, random_state=42)

# Display the chosen sample
random_failed_catheters_mgh[column_names]
chosen_failed_catheters_mgh_encounter_ids = ["3268447806", "3396191507", "3258959083", "3581696894", "3271964781", "3583787789", "3402989492", "3476124055", "3304131417", "3522418740"]
chosen_failed_catheter_mgh_note_ids = ['2903598031', '6426160113', '2535157730', '11282242570',
       '3002237621', '11340428769', '6612736939', '8559605944',
       '3947063203', '9788012155']

In [14]:
df[df['anes_procedure_encounter_id_2273'].isin(chosen_failed_catheters_mgh_encounter_ids)][column_names]

,epic_pmrn,best_timestamp,failed_catheter,true_procedure_type,NotePurposeDSC,Regulated_Anesthesiologist_Name,Regulated_Resident_Name,anes_procedure_encounter_id_2273,anes_procedure_note_id_2260,subsequent_proof_of_failure_note_id
98645,10029685723,2019-09-02 18:01:00+00:00,True,epidural,ADDENDUM,Jingping Wang,Aubrey Samost-Willia,3258959083,2534383247,"['2535157730', '2535640891', '2536024256', '25..."
98644,10029685723,2019-09-03 11:37:00+00:00,True,epidural,ADDENDUM,Lisa Leffert,Daewoong Lee,3258959083,2535157730,"['2535640891', '2536024256', '2536182051']"
98641,10029685723,2019-09-03 17:33:00+00:00,True,epidural,NORMAL,May Pian-Smith,Daewoong Lee,3258959083,2535640891,"['2536024256', '2536182051']"
98643,10029685723,2019-09-04 02:47:00+00:00,True,cse,NORMAL,Bhargavi Krishnan,Aubrey Samost-Willia,3258959083,2536024256,['2536182051']
98642,10029685723,2019-09-04 06:03:00+00:00,False,cse,NORMAL,Bhargavi Krishnan,Aubrey Samost-Willia,3258959083,2536182051,None
46821,10132333898,2019-11-08 16:34:00+00:00,True,cse,NORMAL,Rebecca Minehart,NaN,3268447806,2901995369,"['2903598031', '2905158941']"
46820,10132333898,2019-11-08 19:51:00+00:00,True,epidural,NORMAL,Rebecca Minehart,NaN,3268447806,2903598031,['2905158941']
46819,10132333898,2019-11-09 08:02:00+00:00,False,epidural,NORMAL,Jaime Daly,Michael Fettiplace,3268447806,2905158941,None
15623,10051071404,2019-11-27 03:50:00+00:00,True,epidural,ADDENDUM,Kate Cohen,Rupeng Li,3271964781,3002237621,['3002752923']
15622,10051071404,2019-11-27 08:06:00+00:00,False,epidural,NORMAL,NaN,Ronnie Ibrahim,3271964781,3002752923,None


In [15]:
df[df['anes_procedure_note_id_2260'].isin(chosen_failed_catheter_mgh_note_ids)][column_names]

,epic_pmrn,best_timestamp,failed_catheter,true_procedure_type,NotePurposeDSC,Regulated_Anesthesiologist_Name,Regulated_Resident_Name,anes_procedure_encounter_id_2273,anes_procedure_note_id_2260,subsequent_proof_of_failure_note_id
98644,10029685723,2019-09-03 11:37:00+00:00,True,epidural,ADDENDUM,Lisa Leffert,Daewoong Lee,3258959083,2535157730,"['2535640891', '2536024256', '2536182051']"
46820,10132333898,2019-11-08 19:51:00+00:00,True,epidural,NORMAL,Rebecca Minehart,NaN,3268447806,2903598031,['2905158941']
15623,10051071404,2019-11-27 03:50:00+00:00,True,epidural,ADDENDUM,Kate Cohen,Rupeng Li,3271964781,3002237621,['3002752923']
64344,10054742324,2020-06-22 08:49:00+00:00,True,epidural,NORMAL,Dan Ellis,Peter Ochieng,3304131417,3947063203,['3947625413']
46899,10086456885,2021-11-09 10:34:00+00:00,True,epidural,NORMAL,NaN,Yasmin Fatemi,3396191507,6426160113,['6431936120']
1145,10141675321,2021-12-13 04:15:00+00:00,True,epidural,NORMAL,Emily Naoum,Lukas Matern,3402989492,6612736939,['6612902724']
76461,10047667596,2022-12-11 08:12:00+00:00,True,epidural,NORMAL,Dan Ellis,Allison Dorogi,3476124055,8559605944,"['8561892026', '8566718755']"
31643,10140676171,2023-07-17 14:22:00+00:00,True,epidural,NORMAL,Hilary Gallin,Ryan Norman,3522418740,9788012155,['9789776533']
34330,10031691628,2024-03-30 23:01:00+00:00,False,epidural,ADDENDUM,Gregory Ginsburg,Carl Pierre,3581696894,11282242570,None
151746,10149886797,2024-04-08 14:57:00+00:00,False,epidural,ADDENDUM,Vilma Ortiz,Carl Pierre,3583787789,11340428769,None


In [16]:
random_failed_catheters_mgh[column_names]

,epic_pmrn,best_timestamp,failed_catheter,true_procedure_type,NotePurposeDSC,Regulated_Anesthesiologist_Name,Regulated_Resident_Name,anes_procedure_encounter_id_2273,anes_procedure_note_id_2260,subsequent_proof_of_failure_note_id
43390,10057174186,2021-10-31 18:26:00+00:00,True,cse,ADDENDUM,Rebecca Minehart,NaN,3394126543,6384720411,['6387864829']
6900,10077286655,2019-11-26 00:04:00+00:00,True,epidural,NORMAL,NaN,Ronnie Ibrahim,3271770701,2995596863,['2995643131']
100928,10040215591,2019-10-13 15:47:00+00:00,True,epidural,NORMAL,Lisa Leffert,Raissa Nobrega,3264893822,2777094518,['2779161826']
49210,10093236296,2022-03-20 02:43:00+00:00,True,epidural,NORMAL,NaN,Christopher Kurian,3421693556,7122495394,['7123303814']
51679,10071807795,2018-09-26 07:04:00+00:00,True,epidural,NORMAL,Aalok Agarwala,Christopher Rosicki,3211226326,2074278803,['2074530555']
105895,10093190535,2018-06-01 01:20:00+00:00,True,epidural,NORMAL,Tong-Yan Chen,Christopher Rosicki,3195625915,1921265936,['1921977786']
128555,10022485857,2017-07-14 11:30:00+00:00,True,epidural,ADDENDUM,Jingping Wang,Rabeea Khan,3156646061,1515002443,['1515316759']
76242,10021005466,2021-09-10 12:36:00+00:00,True,epidural,NORMAL,Emily Naoum,Kyle Riley,3384505207,6102043399,['6107479789']
94875,10050909711,2017-08-18 03:50:00+00:00,True,epidural,ADDENDUM,May Pian-Smith,Jamie Sparling,3160718853,1555761733,"['1556957075', '1557052657']"
27514,10068233641,2017-12-04 23:29:00+00:00,True,epidural,ADDENDUM,Vilma Ortiz,Aubrey Samost-Willia,3173736884,1690754065,['1690845881']


# Look at cases where a failure is replaced by the same anesthesia team

Encounter_ID 3607123568 is an example where the same attending/resident team did the index procedure and the replacement (in this case because the epidural migrated out)

In [11]:
def check_if_replacement_has_same_anesthesia_team(row, df):
    if row['is_neuraxial_catheter'] and row['failed_catheter']:
        this_anesthesiologist = row['Regulated_Anesthesiologist_Name']
        this_resident = row['Regulated_Resident_Name']
        subsequent_procedures = row['subsequent_proof_of_failure_note_id'].replace('\'','').replace('[','').replace(']','').split(',')
        for note_id in subsequent_procedures:
            replacing_anesthesiologist = df[(df['anes_procedure_note_id_2260'] == note_id) & (df['is_neuraxial_catheter'] == True)]['Regulated_Anesthesiologist_Name']
            replacing_resident = df[(df['anes_procedure_note_id_2260'] == note_id) & (df['is_neuraxial_catheter'] == True)]['Regulated_Resident_Name']
            if (this_anesthesiologist == replacing_anesthesiologist).any() and (this_resident == replacing_resident).any():
                # print()
                # print('new note')
                # print(this_anesthesiologist)
                # print(this_resident)
                # print(replacing_anesthesiologist)
                # print(replacing_resident)
                # print(this_anesthesiologist == replacing_anesthesiologist)
                # print(this_resident == replacing_resident)
                row['replaced_by_same_team'] = True


In [12]:
df[df['failed_catheter']].apply(lambda x: check_if_replacement_has_same_anesthesia_team(x, df), axis=1)

KeyboardInterrupt: 